In [272]:
import os 
import plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import math 
import re
# import dask.dataframe as dd

In [273]:
os.chdir("C:\\Users\\abhin\\OneDrive\\Documents\\Technical Task")
data = pd.read_csv("match_data.csv")
data.describe()


,Time (s),Pitch_x,Pitch_y,Speed (m/s)
count,985274.000000,985274.000000,985274.000000,985274.000000
mean,3326.746144,-7.420383,-13.591040,1.114658
std,1946.573030,22.886295,25.264996,1.510692
min,0.000000,-86.957029,-51.186749,0.000000
25%,1591.400000,-20.181407,-38.526146,0.044000
50%,3421.050000,-11.946156,-17.893181,0.646000
75%,5053.400000,10.406046,7.338383,1.603500
max,6597.000000,122.893069,158.132620,41.762310


In [274]:
data['Time (s)'] = round(data['Time (s)'], 1)
data['Time (s)'].describe()

count    985274.000000
mean       3326.746144
std        1946.573030
min           0.000000
25%        1591.400000
50%        3421.050000
75%        5053.400000
max        6597.000000
Name: Time (s), dtype: float64

In [275]:

unique_vals = data['participation_id'].unique()
id_list = [id for id in unique_vals if not bool(re.search("ball", id))]

# print("Number of Players represented is: " + str(len(id_list)))

i = 0
players_list = ["Player" + str(i + 1) for i, id in enumerate(id_list)]

player_id_dict = dict(zip(id_list, players_list))

data['player_id'] =  data['participation_id'].map(player_id_dict)


In [276]:
data.head()

,participation_id,Time (s),Pitch_x,Pitch_y,Speed (m/s),player_id
0,455426a2-5604-4c3a-8b89-ed0519f8d77a,0.0,-18.977667,-38.511021,0.060,Player1
1,685d721f-fb27-4fb6-a327-6c8b420bf47f,0.0,-18.740459,-37.195089,0.039,Player2
2,8d6885e6-bfa9-4561-820c-f80eac8d7f62,0.0,-14.467439,-38.767767,0.036,Player3
3,455426a2-5604-4c3a-8b89-ed0519f8d77a,0.1,-18.977667,-38.511021,0.071,Player1
4,685d721f-fb27-4fb6-a327-6c8b420bf47f,0.1,-18.737733,-37.179963,0.019,Player2


In [277]:

unique_vals = data['participation_id'].unique()

id_list = [id for id in unique_vals if not bool(re.search("ball", id))]

print("Number of Players represented is: " + str(len(id_list)))

i = 0
players_list = ["Player" + str(i + 1) for i, id in enumerate(id_list)]

Number of Players represented is: 16


In [278]:


player_data = {}

for player in players_list:
    player_data[str(player)] = data[data['player_id'] == player]




Assign different sprints to gps tracked data which don't justify being subsequent

In [279]:
def Assign_Sprints(player):
    player_temp = player
    current_sprint = [1] * len(player)
    index = 1
    
    for i in range(1, len(player_temp)):
        if player_temp.iloc[i]['Time (s)'] - player_temp.iloc[i-1]['Time (s)'] >= 0.2:
            index += 1
        current_sprint[i] = index
        
        
    player_temp['Sprint'] = current_sprint
    player_temp['Sprint'] = player_temp['Sprint'].astype('object')
        
    return player_temp


In [280]:
for player in players_list:
    player_data[player] = Assign_Sprints(player_data[player])

C:\Users\abhin\AppData\Local\Temp\ipykernel_5196\1371852798.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_temp['Sprint'] = current_sprint
C:\Users\abhin\AppData\Local\Temp\ipykernel_5196\1371852798.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_temp['Sprint'] = player_temp['Sprint'].astype('object')
C:\Users\abhin\AppData\Local\Temp\ipykernel_5196\1371852798.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Aggregate Players Based on Sprint Data

Function to Extract Sprints which have atleast 3 consecutive observations (0.3 seconds)

In [296]:
def extract_sprint_data(player, N):
    sprints_desired = player['Sprint'].value_counts()[lambda x: x >= N]
    return(player[player['Sprint'].isin(sprints_desired.index)])

   
        

In [301]:
obs = {}
for player in players_list:
    obs[player] = len(player_data[player])
obs = dict(sorted(obs.items(), key=lambda item: item[1], reverse = True))
print(obs)

{'Player1': 65971, 'Player14': 64576, 'Player3': 64272, 'Player2': 63962, 'Player4': 63935, 'Player15': 59852, 'Player8': 57684, 'Player10': 57621, 'Player13': 57442, 'Player11': 57395, 'Player5': 57304, 'Player7': 56082, 'Player12': 52860, 'Player16': 47967, 'Player9': 47015, 'Player6': 46104}


Total Time Running on Pitch: Only counting those actions which involve a continuous run of atleast 1 minute (600 ms)

In [303]:
time_leaderboard = {}
for player in players_list:
    
    time_leaderboard[player] = len(extract_sprint_data(player_data[player], 600))/600

time_leaderboard = dict(sorted(time_leaderboard.items(), key=lambda item: item[1], reverse = True))

time_leaderboard

{'Player1': 109.95166666666667,
 'Player14': 107.51666666666667,
 'Player3': 107.08,
 'Player2': 106.46833333333333,
 'Player4': 106.245,
 'Player8': 96.08,
 'Player10': 96.015,
 'Player13': 95.57833333333333,
 'Player5': 95.34,
 'Player11': 94.60333333333334,
 'Player12': 87.89666666666666,
 'Player7': 85.57166666666667,
 'Player15': 83.95166666666667,
 'Player6': 76.085,
 'Player16': 53.44,
 'Player9': 51.931666666666665}

Total Distance Covered by Each Player: Only those runs where it indicates of atleast a minute continously are counted.

In [322]:
def Total_Distance_LeaderBoard(player):
    run = extract_sprint_data(player_data[player], 600)
    dist = 0
    for sprint in run['Sprint'].unique():
        sprint_data = run[run['Sprint'] == sprint]
        for i in range(len(sprint_data)-1):
            dist += math.sqrt((sprint_data.iloc[i+1]["Pitch_x"] - sprint_data.iloc[i]["Pitch_x"])**2 + (sprint_data.iloc[i+1]["Pitch_y"] - sprint_data.iloc[i]["Pitch_y"])**2)
    
    return dist
    


                    

In [323]:
distance_leaderboard = {}
for player in players_list:
    distance_leaderboard[player] = Total_Distance_LeaderBoard(player)


print(distance_leaderboard)


{'Player1': 2849.10344851448, 'Player2': 4717.202753694732, 'Player3': 2263.3772338258555, 'Player4': 2485.8215132410555, 'Player5': 11511.365999526794, 'Player6': 8110.86624029054, 'Player7': 8823.726475040054, 'Player8': 11609.41255341729, 'Player9': 3956.231234577059, 'Player10': 11504.626172822784, 'Player11': 10251.528493340998, 'Player12': 9061.48423940073, 'Player13': 12189.5811254468, 'Player14': 5613.442167735119, 'Player15': 4588.82262023078, 'Player16': 4855.414690967495}


Counting Number of Seconds a Player's instantaneous speed is above Zone 5:

In [332]:
def zone5_count(player):
    run = extract_sprint_data(player_data[player], 10)
    dist_zone5 = 0.0
    for sprint in run['Sprint'].unique():
        sprint_data = run[run['Sprint'] == sprint]
        dist_sprint = 0.0
        for i in range(len(sprint_data)-9): # Window of 10??
            dist_sprint += math.sqrt((sprint_data.iloc[i+1]["Pitch_x"] - sprint_data.iloc[i]["Pitch_x"])**2 + (sprint_data.iloc[i+1]["Pitch_y"] - sprint_data.iloc[i]["Pitch_y"])**2) 
        if(dist_sprint >= 19.8/3.6 and dist_sprint <= 25.1/3.6):
            dist_zone5 += dist_sprint
            
    return dist_zone5


In [333]:
distance_zone5_leaderboard = {}

for player in players_list:
    distance_zone5_leaderboard[player] = zone5_count(player)



print(distance_leaderboard)


{'Player1': 2849.10344851448, 'Player2': 4717.202753694732, 'Player3': 2263.3772338258555, 'Player4': 2485.8215132410555, 'Player5': 11511.365999526794, 'Player6': 8110.86624029054, 'Player7': 8823.726475040054, 'Player8': 11609.41255341729, 'Player9': 3956.231234577059, 'Player10': 11504.626172822784, 'Player11': 10251.528493340998, 'Player12': 9061.48423940073, 'Player13': 12189.5811254468, 'Player14': 5613.442167735119, 'Player15': 4588.82262023078, 'Player16': 4855.414690967495}


In [334]:
print(distance_zone5_leaderboard)

{'Player1': 0.0, 'Player2': 0.0, 'Player3': 0.0, 'Player4': 0.0, 'Player5': 5.682304001567916, 'Player6': 0.0, 'Player7': 12.027518530698101, 'Player8': 6.549492794120983, 'Player9': 129.20382962541373, 'Player10': 0.0, 'Player11': 0.0, 'Player12': 0.0, 'Player13': 0.0, 'Player14': 0.0, 'Player15': 43.37443368899486, 'Player16': 54.81168831775435}


In [63]:
data.columns

Index(['participation_id', 'Time (s)', 'Pitch_x', 'Pitch_y', 'Speed (m/s)',
       'player_id'],
      dtype='object')

In [64]:
for i in player_data.keys():
    print(min(player_data[i]['Time (s)']), max(player_data[i]['Time (s)']))

0.0 6597.0
0.0 6596.9
0.0 6597.0
0.2 6597.0
5.2 6597.0
5.3 5471.9
5.8 6596.9
6.1 6597.0
13.0 6597.0
14.1 6596.9
15.1 6597.0
15.1 6135.8
21.6 6596.9
35.1 6596.9
89.8 6596.9
388.0 6597.0


In [ ]:

temp = intense_sprint(player_data['455426a2-5604-4c3a-8b89-ed0519f8d77a'])
temp

In [65]:
def aggregate_data(player, units):
    if(units == 'seconds'):
        player_agg = player[::10]
        max_speed = player['Speed (m/s)'].rolling(window=10, min_periods=1).max()
        player_agg['Max Speed'] = max_speed[::10]
        return player_agg
    

In [70]:
def distance_covered(player):
    dist = 0
    for i in range(1, len(player)-1):
        dist += math.sqrt((player.iloc[i+1]["Pitch_x"] - player.iloc[i]["Pitch_x"])**2 + (player.iloc[i+1]["Pitch_y"] - player.iloc[i]["Pitch_y"])**2)
    return dist
        

In [77]:
Distance_Covered.append({"d",3})

AttributeError: 'DataFrame' object has no attribute 'append'

In [84]:
Distance_Covered = pd.DataFrame(columns = ['Player', 'Distance Covered'])

for player in player_data.keys():
    Distance_Covered = pd.concat([Distance_Covered, pd.DataFrame([{'Player': player, 'Distance Covered': distance_covered(player_data[player])}])], ignore_index=True)

C:\Users\abhin\AppData\Local\Temp\ipykernel_5196\876618897.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Distance_Covered = pd.concat([Distance_Covered, pd.DataFrame([{'Player': player, 'Distance Covered': distance_covered(player_data[player])}])], ignore_index=True)


In [85]:
Distance_Covered

,Player,Distance Covered
0,Player1,2849.103449
1,Player2,4756.569165
2,Player3,2281.156803
3,Player4,2552.483905
4,Player5,11615.090777
5,Player6,8268.799367
6,Player7,10057.893467
7,Player8,11738.435158
8,Player9,8846.314019
9,Player10,11517.502108


In [98]:
player_data['Player4'].head(20)


,participation_id,Time (s),Pitch_x,Pitch_y,Speed (m/s),player_id
9,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.2,-23.879326,-15.007103,0.03900,Player4
13,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.3,-23.909369,-15.173822,0.03400,Player4
17,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.4,-24.387252,-15.285461,0.00700,Player4
21,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.5,-24.417297,-15.452194,0.05200,Player4
25,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.6,-23.996767,-15.658821,0.04600,Player4
29,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.7,-24.466462,-15.725030,0.03800,Player4
33,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.8,-24.493776,-15.876604,0.08100,Player4
37,b2d99dc2-6919-4c70-b381-6f89ca52048d,0.9,-24.507432,-15.952392,0.06625,Player4
41,b2d99dc2-6919-4c70-b381-6f89ca52048d,1.0,-24.523820,-16.043336,0.05150,Player4
45,b2d99dc2-6919-4c70-b381-6f89ca52048d,1.1,-24.551134,-16.194910,0.03000,Player4


In [313]:
def find_windows(data, threshold):
  """
  Finds all windows of size greater than a minimum size where all values are greater than a threshold.

  Args:
      data: A list of numerical values.
      threshold: The minimum value for a window to be considered.
      min_size: The minimum size of a qualifying window (exclusive).

  Returns:
      A list of tuples representing the starting and ending indices of the windows.
  """
  windows = []
  start = 0
  min_size = 3  # Minimum window size (exclusive)
  for i, value in enumerate(data):
    if value <= threshold:
      # End of a potential window
      if i - start >= min_size:
        windows.append((start, i - 1))
      start = i + 1
    elif i == len(data) - 1 and i - start >= min_size:
      # Handle the last window if it meets the criteria
      windows.append(data[start, i])
  return windows

# Sample data
data = [7, 9, 12, 5, 8, 10, 13, 4, 11, 14]
threshold = 6

# Find windows
windows = find_windows(data, threshold)

# Print the windows
print(windows)

[(0, 2), (4, 6)]
